In [86]:
import numpy as np
import pandas as pd

In [87]:
import random


parametros = {"perfil_1":{"media_1":50,"desviacion_1":1,
                         "media_2":50,"desviacion_2":1,
                         "media_3":50,"desviacion_3":1},
             "perfil_2":{"media_1":10,"desviacion_1":1,
                         "media_2":10,"desviacion_2":1,
                         "media_3":10,"desviacion_3":1},
              "perfil_3":{"media_1":90,"desviacion_1":1,
                         "media_2":90,"desviacion_2":1,
                         "media_3":90,"desviacion_3":1}
             }
perfiles = ["perfil_1","perfil_2","perfil_3"]
observaciones = []
perfil_obs = []
for idx in range(1000):
    perfil = random.sample(perfiles,1)
    perfil = perfil[0]
    perfil_obs.append(perfil)
    obs_1 = random.normalvariate(parametros[perfil]["media_1"], parametros[perfil]["desviacion_1"])
    obs_2 = random.normalvariate(parametros[perfil]["media_2"], parametros[perfil]["desviacion_2"])
    obs_3 = random.normalvariate(parametros[perfil]["media_3"], parametros[perfil]["desviacion_3"])

    observaciones.append([obs_1, obs_2, obs_3])

def conecta_vecinos_graph(vecinos,observaciones):
    from sklearn.neighbors import NearestNeighbors
    import numpy as np

    X = np.array(observaciones)
    nbrs = NearestNeighbors(n_neighbors=vecinos, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(X)

    vecinos_cercanos_graph = {}
    for index in indices:
        vecinos_cercanos_graph[index[0]] = set(index[1:])

    return vecinos_cercanos_graph

graph_vecinos = conecta_vecinos_graph(5,observaciones)

def bfs(grafo, nodo0):
    visitado, cola = set(), [nodo0]
    while cola:
        vertex = cola.pop(0)
        if vertex not in visitado:
            visitado.add(vertex)
            if vertex not in grafo:
                grafo[vertex] = set()
                #print 'Tupla:', vertex, ',', len(grafo[vertex]), ',', len(visitado)
            cola.extend(grafo[vertex] - visitado)

    return visitado

def searchKNN(grafo, nodo0):
    klts = []
    faltantes = grafo
    visitados = bfs(grafo, nodo0)
    c=0
    while len(faltantes)>0:
        kls = {}

        for visitado in list(visitados):
            if grafo.has_key(visitado):
                kls[visitado]= grafo[visitado] #guardamos en el cluster
                del faltantes[visitado]
                visitados.remove(visitado)


        klts.append(kls)


        if len(faltantes) >0:
            #print faltantes.keys()[0]
            #print faltantes
            visitados = bfs(faltantes, faltantes.keys()[0])
    return klts

def create_clusterDict(cluster):
    return dict([('cluster_{0}'.format(clu),clusters[clu]) for clu in range(len(clusters))])

def get_joins(dClustered):
    from itertools import chain
    sso = dClustered.items()
    _presup = [[set(sso[idk][1].keys()).intersection(set(sso[idj][1].keys()))for idj in xrange(idk+1,len(sso)) if idj != idk ] for idk, k in enumerate(sso)]    
    return dict([('cluster_{0}'.format(idp),p) for idp, p in enumerate( list(chain(*_presup))) if p!=set()])
    

def create_adMatrix(clustr_dict):
    ##Numero de clusters, estos vienen en formato de diccionario
    num_clusters = len(clustr_dict)
    
    ##Creamos una matriz de adyacencia con puros ceros
    matrix_inicial = np.zeros((num_clusters, num_clusters))
    names = [cluster for cluster in clustr_dict.keys()]
    matrix_distances = pd.DataFrame(matrix_inicial, index=names, columns=names)
    #matrix_distances
    
    ##Hacemos una comparacion entre cada cluster, para sacar el numero de elementos que comparten, quiza no es lo más
    ##eficiente pero es muy sencillo

    for idx_1 in clustr_dict.keys():
        for idx_2 in clustr_dict.keys():
            if idx_1 != idx_2:
                #Elementos en cada cluster
                elementos_uno = float(len(clustr_dict[idx_1]))
                elementos_dos = float(len(clustr_dict[idx_2]))

                #El cluster con el menor numero de elementos
                minimo_elementos = min(elementos_uno, elementos_dos)

                #cuantos elementos comparten cada cluster
                elementos_compartidos = len(list(set(clustr_dict[idx_1]).intersection(clustr_dict[idx_2])))

                #Asignamos a la matriz la distancia como 1 - num_elemntos_comp / minim_elementos
                matrix_distances[idx_1][idx_2] = 1- (elementos_compartidos/minimo_elementos) 
                #matrix_distances[idx_1][idx_2] = elementos_compartidos
    
    return matrix_distances
    

In [88]:
graph_vecinos = conecta_vecinos_graph(8, observaciones)
clusters=searchKNN(graph_vecinos, 0)
cdict = create_clusterDict(clusters)
joind = get_joins(cdict)
adyM = create_adMatrix(joind)
adyM

,cluster_38,cluster_68,cluster_26,cluster_138,cluster_9,cluster_50,cluster_112,cluster_111,cluster_117,cluster_131,...,cluster_136,cluster_36,cluster_79,cluster_31,cluster_76,cluster_52,cluster_148,cluster_107,cluster_100,cluster_88
cluster_38,0.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,...,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.000000
cluster_68,1.0,0.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,...,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.000000
cluster_26,1.0,1.0,0.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,1.0,...,0.714286,1.0,1.000000,0.0,1.0,1.000000,1.0,1.0,1.000000,1.000000
cluster_138,1.0,1.0,1.000000,0.000000,1.0,1.000000,0.666667,0.666667,1.0,1.0,...,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.000000
cluster_9,1.0,1.0,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.0,1.0,...,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.000000
cluster_50,1.0,1.0,1.000000,1.000000,1.0,0.000000,1.000000,1.000000,1.0,1.0,...,1.000000,1.0,1.000000,1.0,1.0,0.333333,1.0,1.0,1.000000,0.333333
cluster_112,1.0,1.0,1.000000,0.666667,1.0,1.000000,0.000000,0.800000,1.0,1.0,...,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.000000
cluster_111,1.0,1.0,1.000000,0.666667,1.0,1.000000,0.800000,0.000000,1.0,1.0,...,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.000000
cluster_117,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,0.0,1.0,...,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0,1.000000,1.000000
cluster_131,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.0,0.0,...,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,0.5,0.500000,1.000000


In [166]:
#tabler = np.array([])
#for i in range(adyM.shape[0]):
#    for j in xrange(i+1, adyM.shape[0]):
#        tabler.append((i,j,np.array(adyM)[i,j]))

#print adyM
import pandas as pd
import re
adf= pd.DataFrame(adyM)
adf['nodoOrigen']= adf.index
adf_melt= pd.melt(adf,id_vars=['nodoOrigen'],value_vars=list(adf.columns.values)[:-1],var_name='nodoDestino', value_name='valor')
adf_melt= adf_melt[0:225]

#adf_melt['val'] = [str(v) for v in adf_melt['valor']]

adf_melt['orig']= [int(re.sub('^.*_','',n)) for n in adf_melt['nodoOrigen']]
adf_melt['dest']= [int(re.sub('^.*_','',n)) for n in adf_melt['nodoDestino']]
adf_proc= adf_melt[['orig','dest','valor']]

print(adf_proc)
#adf.columns
#adyM.columns
#pandas.melt(adyM,id_vars=)
#pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])


     orig  dest  valor
0      38    38    0.0
1      68    38    1.0
2      26    38    1.0
3     138    38    1.0
4       9    38    1.0
5      50    38    1.0
6     112    38    1.0
7     111    38    1.0
8     117    38    1.0
9     131    38    1.0
10    134    38    1.0
11    135    38    1.0
12    136    38    1.0
13     36    38    1.0
14     79    38    1.0
15     31    38    1.0
16     76    38    1.0
17     52    38    1.0
18    148    38    1.0
19    107    38    1.0
20    100    38    1.0
21     88    38    1.0
22     38    68    1.0
23     68    68    0.0
24     26    68    1.0
25    138    68    1.0
26      9    68    1.0
27     50    68    1.0
28    112    68    1.0
29    111    68    1.0
..    ...   ...    ...
195   107   117    1.0
196   100   117    1.0
197    88   117    1.0
198    38   131    1.0
199    68   131    1.0
200    26   131    1.0
201   138   131    1.0
202     9   131    1.0
203    50   131    1.0
204   112   131    1.0
205   111   131    1.0
206   117  

In [167]:
import json
adf_proc.to_dict()


{'dest': {0: 38,
  1: 38,
  2: 38,
  3: 38,
  4: 38,
  5: 38,
  6: 38,
  7: 38,
  8: 38,
  9: 38,
  10: 38,
  11: 38,
  12: 38,
  13: 38,
  14: 38,
  15: 38,
  16: 38,
  17: 38,
  18: 38,
  19: 38,
  20: 38,
  21: 38,
  22: 68,
  23: 68,
  24: 68,
  25: 68,
  26: 68,
  27: 68,
  28: 68,
  29: 68,
  30: 68,
  31: 68,
  32: 68,
  33: 68,
  34: 68,
  35: 68,
  36: 68,
  37: 68,
  38: 68,
  39: 68,
  40: 68,
  41: 68,
  42: 68,
  43: 68,
  44: 26,
  45: 26,
  46: 26,
  47: 26,
  48: 26,
  49: 26,
  50: 26,
  51: 26,
  52: 26,
  53: 26,
  54: 26,
  55: 26,
  56: 26,
  57: 26,
  58: 26,
  59: 26,
  60: 26,
  61: 26,
  62: 26,
  63: 26,
  64: 26,
  65: 26,
  66: 138,
  67: 138,
  68: 138,
  69: 138,
  70: 138,
  71: 138,
  72: 138,
  73: 138,
  74: 138,
  75: 138,
  76: 138,
  77: 138,
  78: 138,
  79: 138,
  80: 138,
  81: 138,
  82: 138,
  83: 138,
  84: 138,
  85: 138,
  86: 138,
  87: 138,
  88: 9,
  89: 9,
  90: 9,
  91: 9,
  92: 9,
  93: 9,
  94: 9,
  95: 9,
  96: 9,
  97: 9,
  98: 9,
 

In [158]:
import km
mapper = km.KeplerMapper(verbose=1)

In [164]:
#print(np.array(adf_proc))
projected_data=mapper.fit_transform(np.array(adf_proc))
#projected_data=mapper.fit_transform(adf_proc)
#print(projected_data)
#toponet = mapper.map(projected_data, np.array(adf_melt), nr_cubes=10)
toponet = mapper.map(projected_data, np.array(adf_proc), nr_cubes=10)



# Initialize to use t-SNE with 2 components (reduces data to 2 dimensions). Also note high overlap_percentage.
#mapper = km.KeplerMapper(cluster_algorithm=km.cluster.DBSCAN(eps=0.3, min_samples=15), reducer = km.manifold.TSNE(), nr_cubes=35, overlap_perc=0.9, link_local=False, verbose=2)
# Fit and transform data
#projected_data2 = mapper.fit_transform(projected_data)
#complex = mapper.map(projected_data2, dimension_index=[0,1], dimension_name="t-SNE(2) 2D")


..Projecting data using: sum

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

Mapping on data shaped (225, 1) using dimensions

Creating 100 hypercubes.

created 0 edges and 0 nodes in 0:00:00.023888.


In [160]:
mapper.visualize(toponet, path_html="make_circles_keplermapper_output.html", 
                 title="make_circles(n_samples=5000, noise=0.03, factor=0.3)")


Wrote d3.js graph to 'make_circles_keplermapper_output.html'



Wrote d3.js graph to 'make_circles_keplermapper_output.html'


In [197]:
print(toponet["nodes"])
print(toponet["links"])
print(toponet["meta"])

defaultdict(<type 'list'>, {'0_0_0.0_[0]_[ 0.]': [0, 2, 10, 14, 15, 18, 19, 22, 28, 29]})
defaultdict(<type 'list'>, {})
sum


In [170]:
mijson= {"nodes": [{"color":"1","group":6.7166,"name":1,"tooltip":"Grupo: 1"},{"color":"2","group":4.8168,"name":2,"tooltip":"Grupo: 2"},{"color":"3","group":7.5234,"name":3,"tooltip":"Grupo: 3"},{"color":"4","group":4.6096,"name":4,"tooltip":"Grupo: 4"},{"color":"5","group":8.6032,"name":5,"tooltip":"Grupo: 5"},{"color":"6","group":4.6136,"name":6,"tooltip":"Grupo: 6"},{"color":"7","group":7.0913,"name":7,"tooltip":"Grupo: 7"},{"color":"8","group":6.8154,"name":8,"tooltip":"Grupo: 8"},{"color":"9","group":4.7547,"name":9,"tooltip":"Grupo: 9"},{"color":"10","group":8.5122,"name":10,"tooltip":"Grupo: 10"},{"color":"11","group":8.6964,"name":11,"tooltip":"Grupo: 11"},{"color":"12","group":4.1143,"name":12,"tooltip":"Grupo: 12"},{"color":"13","group":8.6217,"name":13,"tooltip":"Grupo: 13"},{"color":"14","group":8.6219,"name":14,"tooltip":"Grupo: 14"},{"color":"15","group":4.3235,"name":15,"tooltip":"Grupo: 15"},{"color":"16","group":7.6903,"name":16,"tooltip":"Grupo: 16"},{"color":"17","group":6.3935,"name":17,"tooltip":"Grupo: 17"},{"color":"18","group":6.0844,"name":18,"tooltip":"Grupo: 18"},{"color":"19","group":9.6175,"name":19,"tooltip":"Grupo: 19"},{"color":"20","group":5.6998,"name":20,"tooltip":"Grupo: 20"},{"color":"21","group":6.438,"name":21,"tooltip":"Grupo: 21"},{"color":"22","group":9.9165,"name":22,"tooltip":"Grupo: 22"},{"color":"23","group":8.5259,"name":23,"tooltip":"Grupo: 23"},{"color":"24","group":5.2389,"name":24,"tooltip":"Grupo: 24"},{"color":"25","group":6.9207,"name":25,"tooltip":"Grupo: 25"},{"color":"26","group":5.4959,"name":26,"tooltip":"Grupo: 26"},{"color":"27","group":7.5685,"name":27,"tooltip":"Grupo: 27"},{"color":"28","group":4.87,"name":28,"tooltip":"Grupo: 28"},{"color":"29","group":7.4348,"name":29,"tooltip":"Grupo: 29"},{"color":"30","group":6.377,"name":30,"tooltip":"Grupo: 30"}], "links": [{"source":0,"target":1,"value":1},{"source":0,"target":2,"value":1},{"source":0,"target":4,"value":4},{"source":0,"target":6,"value":1},{"source":0,"target":7,"value":1},{"source":0,"target":8,"value":2},{"source":0,"target":11,"value":2},{"source":0,"target":12,"value":1},{"source":0,"target":13,"value":2},{"source":0,"target":15,"value":3},{"source":0,"target":16,"value":1},{"source":0,"target":17,"value":1},{"source":0,"target":19,"value":2},{"source":0,"target":20,"value":1},{"source":0,"target":21,"value":1},{"source":0,"target":22,"value":1},{"source":0,"target":23,"value":2},{"source":0,"target":24,"value":2},{"source":0,"target":26,"value":2},{"source":0,"target":27,"value":1},{"source":0,"target":29,"value":1},{"source":1,"target":0,"value":1},{"source":1,"target":3,"value":1},{"source":1,"target":5,"value":1},{"source":1,"target":6,"value":1},{"source":1,"target":7,"value":1},{"source":1,"target":8,"value":2},{"source":1,"target":9,"value":1},{"source":1,"target":10,"value":1},{"source":1,"target":13,"value":1},{"source":1,"target":14,"value":1},{"source":1,"target":15,"value":3},{"source":1,"target":16,"value":1},{"source":1,"target":17,"value":2},{"source":1,"target":19,"value":1},{"source":1,"target":20,"value":2},{"source":1,"target":21,"value":2},{"source":1,"target":22,"value":1},{"source":1,"target":23,"value":1},{"source":1,"target":24,"value":2},{"source":1,"target":26,"value":1},{"source":1,"target":27,"value":1},{"source":2,"target":0,"value":1},{"source":2,"target":3,"value":1},{"source":2,"target":5,"value":2},{"source":2,"target":6,"value":2},{"source":2,"target":7,"value":1},{"source":2,"target":8,"value":1},{"source":2,"target":9,"value":2},{"source":2,"target":10,"value":1},{"source":2,"target":12,"value":1},{"source":2,"target":13,"value":1},{"source":2,"target":15,"value":1},{"source":2,"target":16,"value":2},{"source":2,"target":17,"value":1},{"source":2,"target":18,"value":2},{"source":2,"target":19,"value":1},{"source":2,"target":20,"value":1},{"source":2,"target":21,"value":1},{"source":2,"target":22,"value":1},{"source":2,"target":26,"value":2},{"source":2,"target":27,"value":2},{"source":2,"target":28,"value":1},{"source":3,"target":1,"value":1},{"source":3,"target":2,"value":1},{"source":3,"target":4,"value":2},{"source":3,"target":5,"value":2},{"source":3,"target":6,"value":1},{"source":3,"target":8,"value":1},{"source":3,"target":9,"value":3},{"source":3,"target":10,"value":1},{"source":3,"target":12,"value":3},{"source":3,"target":13,"value":1},{"source":3,"target":15,"value":3},{"source":3,"target":16,"value":1},{"source":3,"target":22,"value":3},{"source":3,"target":25,"value":1},{"source":3,"target":27,"value":1},{"source":3,"target":29,"value":1},{"source":4,"target":0,"value":4},{"source":4,"target":3,"value":2},{"source":4,"target":5,"value":3},{"source":4,"target":9,"value":2},{"source":4,"target":10,"value":1},{"source":4,"target":12,"value":1},{"source":4,"target":13,"value":1},{"source":4,"target":14,"value":2},{"source":4,"target":15,"value":2},{"source":4,"target":18,"value":2},{"source":4,"target":19,"value":1},{"source":4,"target":20,"value":1},{"source":4,"target":21,"value":1},{"source":4,"target":22,"value":1},{"source":4,"target":23,"value":2},{"source":4,"target":24,"value":1},{"source":4,"target":25,"value":1},{"source":4,"target":26,"value":1},{"source":4,"target":29,"value":1},{"source":5,"target":1,"value":1},{"source":5,"target":2,"value":2},{"source":5,"target":3,"value":2},{"source":5,"target":4,"value":3},{"source":5,"target":7,"value":2},{"source":5,"target":8,"value":1},{"source":5,"target":9,"value":2},{"source":5,"target":10,"value":1},{"source":5,"target":11,"value":1},{"source":5,"target":13,"value":2},{"source":5,"target":14,"value":1},{"source":5,"target":15,"value":1},{"source":5,"target":16,"value":1},{"source":5,"target":18,"value":2},{"source":5,"target":23,"value":1},{"source":5,"target":24,"value":1},{"source":5,"target":25,"value":1},{"source":5,"target":28,"value":3},{"source":5,"target":29,"value":1},{"source":6,"target":0,"value":1},{"source":6,"target":1,"value":1},{"source":6,"target":2,"value":2},{"source":6,"target":3,"value":1},{"source":6,"target":9,"value":1},{"source":6,"target":10,"value":3},{"source":6,"target":11,"value":1},{"source":6,"target":12,"value":1},{"source":6,"target":14,"value":1},{"source":6,"target":15,"value":2},{"source":6,"target":16,"value":2},{"source":6,"target":18,"value":1},{"source":6,"target":19,"value":1},{"source":6,"target":20,"value":2},{"source":6,"target":21,"value":1},{"source":6,"target":23,"value":1},{"source":6,"target":24,"value":1},{"source":6,"target":26,"value":1},{"source":6,"target":27,"value":2},{"source":6,"target":28,"value":2},{"source":6,"target":29,"value":1},{"source":7,"target":0,"value":1},{"source":7,"target":1,"value":1},{"source":7,"target":2,"value":1},{"source":7,"target":5,"value":2},{"source":7,"target":8,"value":1},{"source":7,"target":9,"value":2},{"source":7,"target":11,"value":2},{"source":7,"target":12,"value":1},{"source":7,"target":13,"value":1},{"source":7,"target":14,"value":1},{"source":7,"target":16,"value":2},{"source":7,"target":17,"value":3},{"source":7,"target":18,"value":2},{"source":7,"target":19,"value":2},{"source":7,"target":20,"value":1},{"source":7,"target":24,"value":3},{"source":7,"target":25,"value":3},{"source":7,"target":27,"value":1},{"source":7,"target":28,"value":1},{"source":7,"target":29,"value":1},{"source":8,"target":0,"value":2},{"source":8,"target":1,"value":2},{"source":8,"target":2,"value":1},{"source":8,"target":3,"value":1},{"source":8,"target":5,"value":1},{"source":8,"target":7,"value":1},{"source":8,"target":9,"value":2},{"source":8,"target":10,"value":1},{"source":8,"target":11,"value":2},{"source":8,"target":13,"value":3},{"source":8,"target":14,"value":1},{"source":8,"target":15,"value":2},{"source":8,"target":16,"value":1},{"source":8,"target":18,"value":1},{"source":8,"target":19,"value":1},{"source":8,"target":25,"value":4},{"source":8,"target":26,"value":1},{"source":8,"target":27,"value":1},{"source":8,"target":28,"value":3},{"source":9,"target":1,"value":1},{"source":9,"target":2,"value":2},{"source":9,"target":3,"value":3},{"source":9,"target":4,"value":2},{"source":9,"target":5,"value":2},{"source":9,"target":6,"value":1},{"source":9,"target":7,"value":2},{"source":9,"target":8,"value":2},{"source":9,"target":10,"value":2},{"source":9,"target":11,"value":2},{"source":9,"target":12,"value":1},{"source":9,"target":13,"value":4},{"source":9,"target":14,"value":1},{"source":9,"target":15,"value":2},{"source":9,"target":16,"value":1},{"source":9,"target":19,"value":1},{"source":9,"target":21,"value":1},{"source":9,"target":23,"value":2},{"source":9,"target":25,"value":2},{"source":9,"target":26,"value":1},{"source":9,"target":27,"value":2},{"source":9,"target":29,"value":2},{"source":10,"target":1,"value":1},{"source":10,"target":2,"value":1},{"source":10,"target":3,"value":1},{"source":10,"target":4,"value":1},{"source":10,"target":5,"value":1},{"source":10,"target":6,"value":3},{"source":10,"target":8,"value":1},{"source":10,"target":9,"value":2},{"source":10,"target":11,"value":3},{"source":10,"target":12,"value":1},{"source":10,"target":13,"value":3},{"source":10,"target":14,"value":1},{"source":10,"target":15,"value":1},{"source":10,"target":17,"value":1},{"source":10,"target":20,"value":2},{"source":10,"target":22,"value":3},{"source":10,"target":24,"value":2},{"source":10,"target":26,"value":1},{"source":10,"target":27,"value":2},{"source":10,"target":28,"value":1},{"source":11,"target":0,"value":2},{"source":11,"target":5,"value":1},{"source":11,"target":6,"value":1},{"source":11,"target":7,"value":2},{"source":11,"target":8,"value":2},{"source":11,"target":9,"value":2},{"source":11,"target":10,"value":3},{"source":11,"target":12,"value":1},{"source":11,"target":13,"value":2},{"source":11,"target":15,"value":2},{"source":11,"target":17,"value":1},{"source":11,"target":18,"value":2},{"source":11,"target":19,"value":3},{"source":11,"target":20,"value":1},{"source":11,"target":22,"value":2},{"source":11,"target":23,"value":1},{"source":11,"target":25,"value":2},{"source":11,"target":26,"value":1},{"source":11,"target":27,"value":1},{"source":12,"target":0,"value":1},{"source":12,"target":2,"value":1},{"source":12,"target":3,"value":3},{"source":12,"target":4,"value":1},{"source":12,"target":6,"value":1},{"source":12,"target":7,"value":1},{"source":12,"target":9,"value":1},{"source":12,"target":10,"value":1},{"source":12,"target":11,"value":1},{"source":12,"target":13,"value":1},{"source":12,"target":15,"value":1},{"source":12,"target":16,"value":2},{"source":12,"target":18,"value":1},{"source":12,"target":19,"value":3},{"source":12,"target":20,"value":2},{"source":12,"target":22,"value":1},{"source":12,"target":23,"value":2},{"source":12,"target":24,"value":2},{"source":12,"target":25,"value":1},{"source":12,"target":26,"value":1},{"source":12,"target":27,"value":2},{"source":12,"target":28,"value":1},{"source":13,"target":0,"value":2},{"source":13,"target":1,"value":1},{"source":13,"target":2,"value":1},{"source":13,"target":3,"value":1},{"source":13,"target":4,"value":1},{"source":13,"target":5,"value":2},{"source":13,"target":7,"value":1},{"source":13,"target":8,"value":3},{"source":13,"target":9,"value":4},{"source":13,"target":10,"value":3},{"source":13,"target":11,"value":2},{"source":13,"target":12,"value":1},{"source":13,"target":14,"value":2},{"source":13,"target":15,"value":1},{"source":13,"target":17,"value":1},{"source":13,"target":19,"value":2},{"source":13,"target":20,"value":3},{"source":13,"target":21,"value":1},{"source":13,"target":22,"value":2},{"source":13,"target":23,"value":2},{"source":13,"target":25,"value":1},{"source":13,"target":26,"value":2},{"source":13,"target":27,"value":1},{"source":13,"target":29,"value":1},{"source":14,"target":1,"value":1},{"source":14,"target":4,"value":2},{"source":14,"target":5,"value":1},{"source":14,"target":6,"value":1},{"source":14,"target":7,"value":1},{"source":14,"target":8,"value":1},{"source":14,"target":9,"value":1},{"source":14,"target":10,"value":1},{"source":14,"target":13,"value":2},{"source":14,"target":16,"value":1},{"source":14,"target":17,"value":2},{"source":14,"target":18,"value":3},{"source":14,"target":19,"value":2},{"source":14,"target":20,"value":1},{"source":14,"target":21,"value":2},{"source":14,"target":25,"value":1},{"source":14,"target":26,"value":1},{"source":14,"target":29,"value":1},{"source":15,"target":0,"value":3},{"source":15,"target":1,"value":3},{"source":15,"target":2,"value":1},{"source":15,"target":3,"value":3},{"source":15,"target":4,"value":2},{"source":15,"target":5,"value":1},{"source":15,"target":6,"value":2},{"source":15,"target":8,"value":2},{"source":15,"target":9,"value":2},{"source":15,"target":10,"value":1},{"source":15,"target":11,"value":2},{"source":15,"target":12,"value":1},{"source":15,"target":13,"value":1},{"source":15,"target":16,"value":1},{"source":15,"target":17,"value":1},{"source":15,"target":20,"value":1},{"source":15,"target":22,"value":1},{"source":15,"target":23,"value":2},{"source":15,"target":24,"value":1},{"source":15,"target":25,"value":3},{"source":15,"target":26,"value":1},{"source":15,"target":27,"value":4},{"source":15,"target":29,"value":2},{"source":16,"target":0,"value":1},{"source":16,"target":1,"value":1},{"source":16,"target":2,"value":2},{"source":16,"target":3,"value":1},{"source":16,"target":5,"value":1},{"source":16,"target":6,"value":2},{"source":16,"target":7,"value":2},{"source":16,"target":8,"value":1},{"source":16,"target":9,"value":1},{"source":16,"target":12,"value":2},{"source":16,"target":14,"value":1},{"source":16,"target":15,"value":1},{"source":16,"target":17,"value":1},{"source":16,"target":18,"value":1},{"source":16,"target":19,"value":3},{"source":16,"target":21,"value":1},{"source":16,"target":22,"value":2},{"source":16,"target":23,"value":1},{"source":16,"target":25,"value":1},{"source":16,"target":26,"value":2},{"source":16,"target":27,"value":2},{"source":17,"target":0,"value":1},{"source":17,"target":1,"value":2},{"source":17,"target":2,"value":1},{"source":17,"target":7,"value":3},{"source":17,"target":10,"value":1},{"source":17,"target":11,"value":1},{"source":17,"target":13,"value":1},{"source":17,"target":14,"value":2},{"source":17,"target":15,"value":1},{"source":17,"target":16,"value":1},{"source":17,"target":18,"value":1},{"source":17,"target":19,"value":2},{"source":17,"target":20,"value":2},{"source":17,"target":21,"value":1},{"source":17,"target":22,"value":2},{"source":17,"target":23,"value":1},{"source":17,"target":25,"value":1},{"source":17,"target":28,"value":2},{"source":18,"target":2,"value":2},{"source":18,"target":4,"value":2},{"source":18,"target":5,"value":2},{"source":18,"target":6,"value":1},{"source":18,"target":7,"value":2},{"source":18,"target":8,"value":1},{"source":18,"target":11,"value":2},{"source":18,"target":12,"value":1},{"source":18,"target":14,"value":3},{"source":18,"target":16,"value":1},{"source":18,"target":17,"value":1},{"source":18,"target":20,"value":1},{"source":18,"target":26,"value":2},{"source":18,"target":27,"value":1},{"source":18,"target":29,"value":2},{"source":19,"target":0,"value":2},{"source":19,"target":1,"value":1},{"source":19,"target":2,"value":1},{"source":19,"target":4,"value":1},{"source":19,"target":6,"value":1},{"source":19,"target":7,"value":2},{"source":19,"target":8,"value":1},{"source":19,"target":9,"value":1},{"source":19,"target":11,"value":3},{"source":19,"target":12,"value":3},{"source":19,"target":13,"value":2},{"source":19,"target":14,"value":2},{"source":19,"target":16,"value":3},{"source":19,"target":17,"value":2},{"source":19,"target":21,"value":1},{"source":19,"target":22,"value":1},{"source":19,"target":23,"value":2},{"source":19,"target":24,"value":1},{"source":19,"target":27,"value":2},{"source":19,"target":28,"value":1},{"source":19,"target":29,"value":2},{"source":20,"target":0,"value":1},{"source":20,"target":1,"value":2},{"source":20,"target":2,"value":1},{"source":20,"target":4,"value":1},{"source":20,"target":6,"value":2},{"source":20,"target":7,"value":1},{"source":20,"target":10,"value":2},{"source":20,"target":11,"value":1},{"source":20,"target":12,"value":2},{"source":20,"target":13,"value":3},{"source":20,"target":14,"value":1},{"source":20,"target":15,"value":1},{"source":20,"target":17,"value":2},{"source":20,"target":18,"value":1},{"source":20,"target":21,"value":1},{"source":20,"target":22,"value":2},{"source":20,"target":24,"value":1},{"source":20,"target":25,"value":2},{"source":20,"target":26,"value":2},{"source":20,"target":27,"value":2},{"source":20,"target":28,"value":1},{"source":20,"target":29,"value":1},{"source":21,"target":0,"value":1},{"source":21,"target":1,"value":2},{"source":21,"target":2,"value":1},{"source":21,"target":4,"value":1},{"source":21,"target":6,"value":1},{"source":21,"target":9,"value":1},{"source":21,"target":13,"value":1},{"source":21,"target":14,"value":2},{"source":21,"target":16,"value":1},{"source":21,"target":17,"value":1},{"source":21,"target":19,"value":1},{"source":21,"target":20,"value":1},{"source":21,"target":24,"value":2},{"source":21,"target":25,"value":1},{"source":21,"target":26,"value":1},{"source":21,"target":27,"value":1},{"source":21,"target":28,"value":2},{"source":21,"target":29,"value":2},{"source":22,"target":0,"value":1},{"source":22,"target":1,"value":1},{"source":22,"target":2,"value":1},{"source":22,"target":3,"value":3},{"source":22,"target":4,"value":1},{"source":22,"target":10,"value":3},{"source":22,"target":11,"value":2},{"source":22,"target":12,"value":1},{"source":22,"target":13,"value":2},{"source":22,"target":15,"value":1},{"source":22,"target":16,"value":2},{"source":22,"target":17,"value":2},{"source":22,"target":19,"value":1},{"source":22,"target":20,"value":2},{"source":22,"target":23,"value":1},{"source":22,"target":24,"value":2},{"source":22,"target":25,"value":1},{"source":22,"target":26,"value":1},{"source":22,"target":29,"value":2},{"source":23,"target":0,"value":2},{"source":23,"target":1,"value":1},{"source":23,"target":4,"value":2},{"source":23,"target":5,"value":1},{"source":23,"target":6,"value":1},{"source":23,"target":9,"value":2},{"source":23,"target":11,"value":1},{"source":23,"target":12,"value":2},{"source":23,"target":13,"value":2},{"source":23,"target":15,"value":2},{"source":23,"target":16,"value":1},{"source":23,"target":17,"value":1},{"source":23,"target":19,"value":2},{"source":23,"target":22,"value":1},{"source":23,"target":24,"value":1},{"source":23,"target":25,"value":1},{"source":23,"target":26,"value":1},{"source":23,"target":27,"value":1},{"source":23,"target":28,"value":2},{"source":23,"target":29,"value":1},{"source":24,"target":0,"value":2},{"source":24,"target":1,"value":2},{"source":24,"target":4,"value":1},{"source":24,"target":5,"value":1},{"source":24,"target":6,"value":1},{"source":24,"target":7,"value":3},{"source":24,"target":10,"value":2},{"source":24,"target":12,"value":2},{"source":24,"target":15,"value":1},{"source":24,"target":19,"value":1},{"source":24,"target":20,"value":1},{"source":24,"target":21,"value":2},{"source":24,"target":22,"value":2},{"source":24,"target":23,"value":1},{"source":24,"target":26,"value":1},{"source":24,"target":27,"value":1},{"source":24,"target":28,"value":1},{"source":25,"target":3,"value":1},{"source":25,"target":4,"value":1},{"source":25,"target":5,"value":1},{"source":25,"target":7,"value":3},{"source":25,"target":8,"value":4},{"source":25,"target":9,"value":2},{"source":25,"target":11,"value":2},{"source":25,"target":12,"value":1},{"source":25,"target":13,"value":1},{"source":25,"target":14,"value":1},{"source":25,"target":15,"value":3},{"source":25,"target":16,"value":1},{"source":25,"target":17,"value":1},{"source":25,"target":20,"value":2},{"source":25,"target":21,"value":1},{"source":25,"target":22,"value":1},{"source":25,"target":23,"value":1},{"source":25,"target":26,"value":1},{"source":25,"target":28,"value":1},{"source":25,"target":29,"value":2},{"source":26,"target":0,"value":2},{"source":26,"target":1,"value":1},{"source":26,"target":2,"value":2},{"source":26,"target":4,"value":1},{"source":26,"target":6,"value":1},{"source":26,"target":8,"value":1},{"source":26,"target":9,"value":1},{"source":26,"target":10,"value":1},{"source":26,"target":11,"value":1},{"source":26,"target":12,"value":1},{"source":26,"target":13,"value":2},{"source":26,"target":14,"value":1},{"source":26,"target":15,"value":1},{"source":26,"target":16,"value":2},{"source":26,"target":18,"value":2},{"source":26,"target":20,"value":2},{"source":26,"target":21,"value":1},{"source":26,"target":22,"value":1},{"source":26,"target":23,"value":1},{"source":26,"target":24,"value":1},{"source":26,"target":25,"value":1},{"source":26,"target":28,"value":2},{"source":26,"target":29,"value":3},{"source":27,"target":0,"value":1},{"source":27,"target":1,"value":1},{"source":27,"target":2,"value":2},{"source":27,"target":3,"value":1},{"source":27,"target":6,"value":2},{"source":27,"target":7,"value":1},{"source":27,"target":8,"value":1},{"source":27,"target":9,"value":2},{"source":27,"target":10,"value":2},{"source":27,"target":11,"value":1},{"source":27,"target":12,"value":2},{"source":27,"target":13,"value":1},{"source":27,"target":15,"value":4},{"source":27,"target":16,"value":2},{"source":27,"target":18,"value":1},{"source":27,"target":19,"value":2},{"source":27,"target":20,"value":2},{"source":27,"target":21,"value":1},{"source":27,"target":23,"value":1},{"source":27,"target":24,"value":1},{"source":27,"target":28,"value":2},{"source":28,"target":2,"value":1},{"source":28,"target":5,"value":3},{"source":28,"target":6,"value":2},{"source":28,"target":7,"value":1},{"source":28,"target":8,"value":3},{"source":28,"target":10,"value":1},{"source":28,"target":12,"value":1},{"source":28,"target":17,"value":2},{"source":28,"target":19,"value":1},{"source":28,"target":20,"value":1},{"source":28,"target":21,"value":2},{"source":28,"target":23,"value":2},{"source":28,"target":24,"value":1},{"source":28,"target":25,"value":1},{"source":28,"target":26,"value":2},{"source":28,"target":27,"value":2},{"source":28,"target":29,"value":1},{"source":29,"target":0,"value":1},{"source":29,"target":3,"value":1},{"source":29,"target":4,"value":1},{"source":29,"target":5,"value":1},{"source":29,"target":6,"value":1},{"source":29,"target":7,"value":1},{"source":29,"target":9,"value":2},{"source":29,"target":13,"value":1},{"source":29,"target":14,"value":1},{"source":29,"target":15,"value":2},{"source":29,"target":18,"value":2},{"source":29,"target":19,"value":2},{"source":29,"target":20,"value":1},{"source":29,"target":21,"value":2},{"source":29,"target":22,"value":2},{"source":29,"target":23,"value":1},{"source":29,"target":25,"value":2},{"source":29,"target":26,"value":3},{"source":29,"target":28,"value":1}]}
print(mijson)

{'nodes': [{'color': '1', 'group': 6.7166, 'name': 1, 'tooltip': 'Grupo: 1'}, {'color': '2', 'group': 4.8168, 'name': 2, 'tooltip': 'Grupo: 2'}, {'color': '3', 'group': 7.5234, 'name': 3, 'tooltip': 'Grupo: 3'}, {'color': '4', 'group': 4.6096, 'name': 4, 'tooltip': 'Grupo: 4'}, {'color': '5', 'group': 8.6032, 'name': 5, 'tooltip': 'Grupo: 5'}, {'color': '6', 'group': 4.6136, 'name': 6, 'tooltip': 'Grupo: 6'}, {'color': '7', 'group': 7.0913, 'name': 7, 'tooltip': 'Grupo: 7'}, {'color': '8', 'group': 6.8154, 'name': 8, 'tooltip': 'Grupo: 8'}, {'color': '9', 'group': 4.7547, 'name': 9, 'tooltip': 'Grupo: 9'}, {'color': '10', 'group': 8.5122, 'name': 10, 'tooltip': 'Grupo: 10'}, {'color': '11', 'group': 8.6964, 'name': 11, 'tooltip': 'Grupo: 11'}, {'color': '12', 'group': 4.1143, 'name': 12, 'tooltip': 'Grupo: 12'}, {'color': '13', 'group': 8.6217, 'name': 13, 'tooltip': 'Grupo: 13'}, {'color': '14', 'group': 8.6219, 'name': 14, 'tooltip': 'Grupo: 14'}, {'color': '15', 'group': 4.3235, 'na

In [179]:
#mapper.visualize(mijson, path_html="salida_km.html", title="Salida KM")
mapper.visualize(list(adf_proc), "keplermapper_digits_custom_tooltips.html", "Digits", graph_gravity=0.25)


TypeError: list indices must be integers, not str

In [3]:
%load_ext rmagic

c(1,2,3)

The rmagic extension is already loaded. To reload it, use:
  %reload_ext rmagic


NameError: name 'c' is not defined

In [17]:
# Import the class
import km

# Some sample data
from sklearn import datasets
data, labels = datasets.make_circles(n_samples=5000, noise=0.03, factor=0.3)
#datasets.make_circles(n_samples=5000, noise=0.03, factor=0.3)

In [18]:


# Initialize
mapper = km.KeplerMapper(verbose=1)


In [36]:

# Fit to and transform the data
projected_data = mapper.fit_transform(data, projection=[0,1]) # X-Y axis
print(projected_data)
print(data)


..Projecting data using: [0, 1]

..Scaling with: MinMaxScaler(copy=True, feature_range=(0, 1))

[[ 0.40169948  0.03125879]
 [ 0.74438546  0.09590421]
 [ 0.02830464  0.59978935]
 ..., 
 [ 0.35733041  0.94741649]
 [ 0.10529196  0.22274882]
 [ 0.83018643  0.17240929]]
[[-0.20215051 -1.01226173]
 [ 0.52052359 -0.87272601]
 [-0.98958506  0.21489914]
 ..., 
 [-0.29571831  0.9652448 ]
 [-0.82723018 -0.59893466]
 [ 0.70146515 -0.70759144]]


In [39]:
# Create dictionary called 'complex' with nodes, edges and meta-information
complex = mapper.map(projected_data, data, nr_cubes=10)
print(complex)

Mapping on data shaped (5000, 2) using dimensions

Creating 1000 hypercubes.

created 97 edges and 59 nodes in 0:00:08.743501.
Mapping on data shaped (5000, 2) using dimensions

Creating 1000 hypercubes.

created 97 edges and 59 nodes in 0:00:08.908465.


{'links': defaultdict(<type 'list'>, {'0_3_0.0_[0 3]_[ 0.   0.3]': ['0_2_0.0_[0 2]_[ 0.   0.2]', '0_4_0.0_[0 4]_[ 0.   0.4]'], '8_87_0.0_[8 7]_[ 0.8  0.7]': ['9_97_0.0_[9 7]_[ 0.9  0.7]', '8_86_0.0_[8 6]_[ 0.8  0.6]', '8_88_0.0_[8 8]_[ 0.8  0.8]', '9_96_0.0_[9 6]_[ 0.9  0.6]'], '3_34_0.0_[3 4]_[ 0.3  0.4]': ['3_33_0.0_[3 3]_[ 0.3  0.3]', '3_35_0.0_[3 5]_[ 0.3  0.5]', '4_44_0.0_[4 4]_[ 0.4  0.4]', '4_43_0.0_[4 3]_[ 0.4  0.3]'], '0_2_0.0_[0 2]_[ 0.   0.2]': ['1_11_0.0_[1 1]_[ 0.1  0.1]', '1_12_0.0_[1 2]_[ 0.1  0.2]'], '1_11_0.0_[1 1]_[ 0.1  0.1]': ['2_20_0.0_[2 0]_[ 0.2  0. ]', '2_21_0.0_[2 1]_[ 0.2  0.1]', '1_10_0.0_[1 0]_[ 0.1  0. ]', '1_12_0.0_[1 2]_[ 0.1  0.2]'], '2_20_0.0_[2 0]_[ 0.2  0. ]': ['2_21_0.0_[2 1]_[ 0.2  0.1]', '1_10_0.0_[1 0]_[ 0.1  0. ]', '3_30_0.0_[3 0]_[ 0.3  0. ]'], '4_46_0.0_[4 6]_[ 0.4  0.6]': ['5_56_0.0_[5 6]_[ 0.5  0.6]', '3_35_0.0_[3 5]_[ 0.3  0.5]', '3_36_0.0_[3 6]_[ 0.3  0.6]', '5_55_0.0_[5 5]_[ 0.5  0.5]', '4_45_0.0_[4 5]_[ 0.4  0.5]'], '8_81_0.0_[8 1]_[ 0.8 

In [10]:

# Visualize it
mapper.visualize(complex, path_html="make_circles_keplermapper_output.html", 
                 title="make_circles(n_samples=5000, noise=0.03, factor=0.3)")


Wrote d3.js graph to 'make_circles_keplermapper_output.html'


In [47]:
xlinks= {'A':['B','C','D','E'],'B':['C','D'],'C':['B','C','D','E'],'E':['A','F']}
xmeta='[0,1]'
xnodes= {'A':[1,2,3,4,5],'B':[1,6],'C':[7],'D':[8,9,10,11,12,13,14,15],'E':[16,17],'F':[18]}
print(xlinks)
print(xnodes)

xfull= {'links':xlinks,'meta':xmeta,'nodes':xnodes}
print(xfull)

{'A': ['B', 'C', 'D', 'E'], 'C': ['B', 'C', 'D', 'E'], 'B': ['C', 'D'], 'E': ['A', 'F']}
{'A': [1, 2, 3, 4, 5], 'C': [7], 'B': [1, 6], 'E': [16, 17], 'D': [8, 9, 10, 11, 12, 13, 14, 15], 'F': [18]}
{'nodes': {'A': [1, 2, 3, 4, 5], 'C': [7], 'B': [1, 6], 'E': [16, 17], 'D': [8, 9, 10, 11, 12, 13, 14, 15], 'F': [18]}, 'meta': '[0,1]', 'links': {'A': ['B', 'C', 'D', 'E'], 'C': ['B', 'C', 'D', 'E'], 'B': ['C', 'D'], 'E': ['A', 'F']}}


In [52]:

mapper.visualize(xfull, path_html="xtest.html", title="make_circles(n_samples=5000, noise=0.03, factor=0.3)")


Wrote d3.js graph to 'xtest.html'


%%HTML
src="https://yogui.cloudapp.net/tree/xtest.html" width="100%"


<DOCTYPE HTML>
<html>
<head>
</head>
<body>
<iframe src="https://yogui.cloudapp.net/tree/xtest.html" width="100%"></iframe>
</body>
</html>

In [50]:
# Import the class
import km

from sklearn import datasets
data, labels = datasets.make_circles(n_samples=5000, noise=0.03, factor=0.3)

In [58]:
%%HTML
<img src="https://yogui.cloudapp.net/tree/xtest.html" width="100%">